<a href="https://colab.research.google.com/github/alyarctiq/tfblog-inceptionv3/blob/master/colab-notebook/Tensorflow_GCP_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prep 

In [24]:
# Validate Notebook
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
    print("TF Version: ",  tf.__version__)
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Tensorflow and Keras
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import applications
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.inception_v3 import decode_predictions
from tensorflow.keras.preprocessing.image import load_img

# Helper libraries
import os
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from io import BytesIO
import urllib
from matplotlib.pyplot import imshow

tf.keras.backend.clear_session()  # For easy reset of notebook state.

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Note: using Google CoLab
TF Version:  2.4.0
Found GPU at: /device:GPU:0


## Load Inceptionv3 Model

In [25]:
# Load Model
model = InceptionV3(include_top=True, weights='imagenet')

## Load Test Image

In [27]:
def loadImage(URL):
  res = request.urlopen(URL).read()
  img = Image.open(BytesIO(res)).resize((150,150))
  return img

In [28]:
img = loadImage("https://news.harvard.edu/wp-content/uploads/2019/04/iStock-515813368.jpg")

TypeError: ignored

## Pre-process

In [ ]:
w, h = img.size
s = min(w, h)
y = (h - s) // 2
x = (w - s) // 2
img = img.crop((x, y, s, s))
target_size = max(x for x in model.layers[0].input_shape if x)
img = img.resize((299, 299), Image.ANTIALIAS)
imshow(np.asarray(img))

In [ ]:
np_img = image.img_to_array(img)
img_batch = np.expand_dims(np_img, axis=0)
pre_processed = preprocess_input(img_batch)
pre_processed.shape

## Run inference and Decode Output

In [ ]:
features = model.predict(pre_processed)
features.shape

In [ ]:
decode_predictions(features, top=5)

## Export Model in SavedModel format


In [ ]:
# Export Path
export_path = '/content/saved_model/v1/'

# Clean Up previous dir
!rm -rf export_path

# Save the entire model as a SavedModel.
!mkdir -p export_path

#save model
tf.saved_model.save(model, os.path.join('/content/',export_path))

# my_model directory
!ls -l /content/saved_model/v1/

In [ ]:
!ls -R '/content/saved_model/v1/'

In [ ]:
!saved_model_cli show --dir saved_model/v1 --tag_set serve

## GCloud Auth

In [ ]:
!gcloud auth login

In [ ]:
!gcloud config set project tf-blog

## GCS Bucket

In [ ]:
!gsutil mb gs://tf-blog
!gsutil list

In [ ]:
!gsutil -m cp -r saved_model/v1/ gs://tf-blog

In [ ]:
!gsutil ls gs://tf-blog/v1

## Google AI Platform

In [ ]:
!gcloud ai-platform models create tfblog --regions=us-central1

In [ ]:
!gcloud ai-platform versions create v1 --model=tfblog --framework=tensorflow --origin=gs://tf-blog/v1 --python-version=3.5 --runtime-version=1.14

## Google App Engine

In [ ]:
!git clone https://github.com/alyarctiq/tfblog-inceptionv3
!cd  tfblog-inceptionv3/appengine/
!ls -l tfblog-inceptionv3/appengine/

In [ ]:
!gcloud app deploy tfblog-inceptionv3/appengine/

## Zip Model

In [ ]:
#zip and download
from google.colab import files
download_path = 'saved_model/v1.zip'
export_path='saved_model/v1'
!zip -r {download_path} {export_path}

## Save to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!mkdir -p /content/gdrive/"My Drive/colab-download/models"

In [ ]:
!cp -R /content/saved_model/v1.zip /content/gdrive/"My Drive/colab-download/models/"

In [ ]:
## Clean Up CoLab
!rm -rf '/content/saved_model/v1/'